In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
#!pip install LightGBM

start_time = time.time()

path1 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//application_train.csv'
path2 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//application_test.csv'
path3 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//bureau.csv'
path4 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//bureau_balance.csv'
path5 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//credit_card_balance.csv'
path6 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//installments_payments.csv'
path7 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//POS_CASH_balance.csv'
path8 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//previous_application.csv'
path9 = 'C://Users//shan wu//Documents//data science//home credit default risk//data//sample_submission.csv'

df_train = pd.read_csv(path1) #(307511, 122)
df_test = pd.read_csv(path2) #(48744, 121)
df_bureau = pd.read_csv(path3) #(1716428, 17)
df_bureau_balance = pd.read_csv(path4) #(27299925, 3)
df_credit_card_balance = pd.read_csv(path5) #(3840312, 23)
df_installments_payments = pd.read_csv(path6) #(13605401, 8)
df_POS_CASH_balance = pd.read_csv(path7) #(10001358, 8)
df_previous_application = pd.read_csv(path8) #(1670214, 37)
df_sample_submission = pd.read_csv(path9) #(48744, 2)

print("--- %s seconds ---" % (time.time() - start_time))

#df_train.head()
#df_train.shape


--- 54.81419849395752 seconds ---


In [68]:
#Combine files
#df_bureau['CREDIT_TYPE'].unique()
#df_bureau_balance[df_bureau_balance['SK_ID_BUREAU'] == 5715448]

df_bureau_balance['STATUS_val'] = pd.to_numeric(df_bureau_balance['STATUS'],errors='coerce')
df_bureau_balance_grpby = df_bureau_balance.groupby('SK_ID_BUREAU')['STATUS_val'].mean()
df_bureau_balance_grpby = df_bureau_balance_grpby.to_frame()
df_bureau_balance_grpby = df_bureau_balance_grpby.reset_index('SK_ID_BUREAU')
df_bureau_combine = pd.merge(df_bureau, df_bureau_balance_grpby, how='left', on='SK_ID_BUREAU') #, left_on=None, right_on=None
df_bureau_grpby = df_bureau_combine.groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': np.sum, 'AMT_CREDIT_SUM_DEBT': np.sum, 
                                            'AMT_CREDIT_SUM_LIMIT': np.sum, 'AMT_CREDIT_SUM_OVERDUE': np.sum, 
                                            'SK_ID_BUREAU': np.size, 'STATUS_val': np.mean})
df_bureau_grpby = df_bureau_grpby.rename(columns={'AMT_CREDIT_SUM':'AMT_CREDIT_SUM_BUREAU','AMT_CREDIT_SUM_DEBT':'AMT_CREDIT_SUM_DEBT_BUREAU',
                               'AMT_CREDIT_SUM_LIMIT':'AMT_CREDIT_SUM_LIMIT_BUREAU', 'SK_ID_BUREAU':'BUREAU_COUNT',
                                'AMT_CREDIT_SUM_OVERDUE':'AMT_CREDIT_SUM_OVERDUE_BUREAU'})

df_bureau_grpby = df_bureau_grpby.reset_index('SK_ID_CURR')

df_cash_grpby = df_POS_CASH_balance.groupby('SK_ID_CURR').agg({'SK_DPD': np.sum, 'SK_DPD_DEF': np.sum})
df_cash_grpby = df_cash_grpby.rename(columns={'SK_DPD':'SK_DPD_Cash','SK_DPD_DEF':'SK_DPD_DEF_Cash'})
df_cash_grpby = df_cash_grpby.reset_index('SK_ID_CURR')

df_credit_card_grpby = df_credit_card_balance.groupby('SK_ID_CURR').agg({'SK_DPD': np.sum, 'SK_DPD_DEF': np.sum,
                                                                        'AMT_BALANCE': np.mean, 'AMT_CREDIT_LIMIT_ACTUAL': np.mean})
df_credit_card_grpby = df_credit_card_grpby.rename(columns={'SK_DPD':'SK_DPD_CC','SK_DPD_DEF':'SK_DPD_DEF_CC'})
df_credit_card_grpby = df_credit_card_grpby.reset_index('SK_ID_CURR')

df_installments_payments['Diff'] = df_installments_payments['AMT_PAYMENT'] - df_installments_payments['AMT_INSTALMENT']
df_installments_grpby = df_installments_payments.groupby('SK_ID_CURR').agg({'Diff': np.mean})
df_installments_grpby = df_installments_grpby.reset_index('SK_ID_CURR')

df_previous_application['Diff_application'] = df_previous_application['AMT_APPLICATION'] - df_previous_application['AMT_CREDIT']
df_previous_application_grpby = df_previous_application.groupby('SK_ID_CURR').agg({'SK_ID_PREV': np.size, 'Diff_application': np.mean,
                                                                                  'AMT_APPLICATION': np.mean})
df_previous_application_grpby = df_previous_application_grpby.rename(columns={'SK_ID_PREV':'PREV_COUNT'})
df_previous_application_grpby = df_previous_application_grpby.reset_index('SK_ID_CURR')


In [71]:
#df_train['TARGET'].unique()
#df_train['NAME_CONTRACT_TYPE'].unique()
#df_train3 = df_train2.drop(['AMT_CREDIT_SUM_BUREAU','AMT_CREDIT_SUM_DEBT_BUREAU','AMT_CREDIT_SUM_LIMIT_BUREAU','AMT_CREDIT_SUM_OVERDUE_BUREAU','BUREAU_COUNT','STATUS_val'],axis = 1)
#df_test3 = df_test2.drop(['AMT_CREDIT_SUM_BUREAU','AMT_CREDIT_SUM_DEBT_BUREAU','AMT_CREDIT_SUM_LIMIT_BUREAU','AMT_CREDIT_SUM_OVERDUE_BUREAU','BUREAU_COUNT','STATUS_val'],axis = 1)

if 'TARGET' in df_test:
    df_test = df_test.drop(['TARGET'],axis = 1)

df_train2 = pd.merge(df_train, df_bureau_grpby, how='left', on='SK_ID_CURR')
df_test2 = pd.merge(df_test, df_bureau_grpby, how='left', on='SK_ID_CURR')

df_train3 = pd.merge(df_train2, df_cash_grpby, how='left', on='SK_ID_CURR')
df_test3 = pd.merge(df_test2, df_cash_grpby, how='left', on='SK_ID_CURR')

df_train4 = pd.merge(df_train3, df_credit_card_grpby, how='left', on='SK_ID_CURR')
df_test4 = pd.merge(df_test3, df_credit_card_grpby, how='left', on='SK_ID_CURR')

df_train5 = pd.merge(df_train4, df_installments_grpby, how='left', on='SK_ID_CURR')
df_test5 = pd.merge(df_test4, df_installments_grpby, how='left', on='SK_ID_CURR')

df_train6 = pd.merge(df_train5, df_previous_application_grpby, how='left', on='SK_ID_CURR')
df_test6 = pd.merge(df_test5, df_previous_application_grpby, how='left', on='SK_ID_CURR')

y_train = df_train.iloc[:,1]
X_train = df_train6.iloc[:,2:]
X_test = df_test6.iloc[:,1:]

In [81]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMClassifier


le = preprocessing.LabelEncoder()
for i in range(len(X_train.columns)):
    if X_train.iloc[:,i].dtype == 'O':
        X_train.iloc[:,i] = X_train.iloc[:,i].astype(str).fillna('NA')
        X_train.iloc[:,i] = le.fit_transform(X_train.iloc[:,i])
    else:
        X_train.iloc[:,i] = X_train.iloc[:,i].fillna(-1)
#clf = DecisionTreeClassifier(max_depth = 3).fit(X_train, y_train) #0.69
#clf = LogisticRegression(C=100).fit(X_train, y_train) #0.64
#clf = SVC(kernel = 'linear', C=1).fit(X_train, y_train) #couldn't finish
#clf = RandomForestClassifier().fit(X_train, y_train) #0.99
#clf =  RandomForestClassifier(n_estimators = 50 , max_depth=15) #0.94, kaggle score 0.721
#clf =  RandomForestClassifier(n_estimators = 20 , max_depth=15) #0.92, kaggle score 0.706
#clf =  RandomForestClassifier(n_estimators = 50 , max_depth=12) #0.86, kaggle score 0.73
#clf =  RandomForestClassifier(n_estimators = 50 , max_depth=13) #0.89
#clf =  RandomForestClassifier(n_estimators = 50 , max_depth=14) #0.9148
#clf =  RandomForestClassifier(n_estimators = 50 , max_depth=16) #0.956
#clf =  RandomForestClassifier(n_estimators = 50 , max_depth=17) #0.972
#clf =  RandomForestClassifier(n_estimators = 40 , max_depth=15) #0.935
#clf =  RandomForestClassifier(n_estimators = 30 , max_depth=15) #0.932
#clf =  RandomForestClassifier(n_estimators = 20 , max_depth=15) #0.922
#clf =  RandomForestClassifier(n_estimators = 20 , max_depth=12) #0.843
#clf =  RandomForestClassifier(n_estimators = 20 , max_depth=14) #0.903
#clf =  RandomForestClassifier(n_estimators = 20 , max_depth=16) #0.9428
#clf =  RandomForestClassifier(n_estimators = 60 , max_depth=12) #0.867, kaggle score 0.731
#clf =  RandomForestClassifier(n_estimators = 60 , max_depth=12)


#clf = GradientBoostingClassifier().fit(X_train, y_train) #0.75, kaggle score 0.744
#clf = GradientBoostingClassifier(loss='exponential').fit(X_train, y_train) #0.765
#clf = GradientBoostingClassifier(loss='exponential', n_estimators = 120).fit(X_train, y_train) #0.77
#clf = GradientBoostingClassifier(loss='exponential', learning_rate=0.1, n_estimators = 120,subsample = 1).fit(X_train, y_train) #0.769， kaggle score: 0.748

#clf = GradientBoostingClassifier(loss='exponential', learning_rate=0.01, n_estimators = 100,subsample = 0.5).fit(X_train, y_train) #0.7749

clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )
    
#clf = GradientBoostingClassifier(loss='exponential', learning_rate=0.02, n_estimators = 10000,max_depth=8).fit(X_train, y_train) 

#clf = MLPClassifier(solver='lbfgs', activation = 'tanh',alpha = 100,hidden_layer_sizes = [100, 100],random_state = 0).fit(X_train, y_train) #0.
#clf = MLPRegressor(hidden_layer_sizes = [100,100],activation = 'relu',alpha = 100,solver = 'lbfgs').fit(X_train, y_train)

#clf.fit(X_train, y_train)

cv_score = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')

cv_score

0.9619630903800338

In [82]:
pred2 = [row[1] for row in pred]
df_test['TARGET'] = pred2
submission = df_test.loc[:, ['SK_ID_CURR','TARGET']]
path10 = 'C://Users//shan wu//Documents//data science//home credit default risk//submission//final_submission.csv'

submission.to_csv(path10, index=False)

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,SK_ID_BUREAU,STATUS_val
SK_ID_CURR,,,,,,
100001.0,569340.000,0.0,0.0,0.0,4,0.000000
100002.0,383067.000,0.0,0.0,0.0,6,0.364833
100003.0,207400.500,0.0,0.0,0.0,3,NaN
100004.0,189037.800,0.0,0.0,0.0,2,NaN
100005.0,58500.000,0.0,0.0,0.0,1,0.000000
100007.0,146250.000,0.0,0.0,0.0,1,NaN
100008.0,200839.500,0.0,0.0,0.0,2,NaN
100009.0,3273250.500,0.0,0.0,0.0,14,NaN
100010.0,315000.000,0.0,NaN,0.0,1,0.000000


array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [60]:
df_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [77]:

?GradientBoostingClassifier

In [76]:
from lightgbm import LGBMClassifier


ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
#df_bureau & df_bureau_balance
#df_POS_CASH_balance
#df_previous_application.head()
df_credit_card_balance.head()
df_installments_payments.head()
df_previous_application.head()

combine = pd.merge(df_train, df_bureau, how='inner', on='SK_ID_CURR') #, left_on=None, right_on=None
df_previous_application.head()m
#df_bureau_balance.groupby('MONTHS_BALANCE').count()
#df_agg['count'].groupby(level=0, group_keys=False)
#df.sort_values(['job','count'],ascending=False).groupby('job').head(3)